# Import Library

In [2]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import pos_tag

import newspaper
from newspaper import Article

import string
import json
import operator
from collections import Counter

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

In [3]:
import re

# Read Data

In [4]:
header = ["GlobalEventId", "sqldate", "MonthYear","Year",
               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1Type1Code",
               "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2Type1Code",
               "IsRootEvent","EventCode","EventBaseCode","EventRootCode",
               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
               "Actor1Geo_FullName","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long",
               "Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_Lat","Actor2Geo_Long",
               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code","ActionGeo_Lat","ActionGeo_Long",
               "SOURCEURL"]

In [5]:
#ubah path_clean sesuai tempat clean di lokal
#path_clean = '2016-clean.csv'
path_clean = 'E:/digitalent-project/csv/2017-clean.csv'
data_gdelt = pd.read_csv(path_clean, delimiter = ',', encoding = "ISO-8859-1", names=header)
data_gdelt.sample(4)

,GlobalEventId,sqldate,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1Type1Code,Actor2Code,...,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
278814,636062804,20170312,201703,2017,GOV,PRESIDENT,NaN,NaN,GOV,NGA,...,NI,NI11,9.08333,7.53333,"Abuja, Abuja Federal Capital Territory, Nigeria",NI,NI11,9.08333,7.53333,http://www.premiumtimesng.com/news/more-news/2...
1254326,709529462,20171124,201711,2017,USA,UNITED STATES,USA,NaN,NaN,PHL,...,US,USCA,34.09830,-118.32700,"Philippine, Benguet, Philippines",RP,RP10,16.45000,120.55000,http://www.philstar.com/headlines/2017/11/24/1...
850495,679359744,20170807,201708,2017,EDUEDU,UNIVERSITY,NaN,NaN,EDU,NaN,...,NaN,NaN,NaN,NaN,"Islamabad, Islamabad, Pakistan",PK,PK08,33.70000,73.16670,https://www.onenewsnow.com/media/2017/08/07/fa...
223163,631561239,20170227,201702,2017,USA,WASHINGTON,USA,NaN,NaN,GOV,...,IS,IS,31.50000,34.75000,Israel,IS,IS,31.50000,34.75000,http://nation.com.pk/national/27-Feb-2017/us-c...


### Inisialisasi Stopwords

In [6]:
stop_words = set(stopwords.words('english'))

### Inisialisasi Stemming

In [7]:
ps = nltk.stem.PorterStemmer()

### Inisialisasi dict res untuk menampung result

In [8]:
res = {}

In [9]:
import time

In [14]:
from nltk.tag import pos_tag

# Fungsi Scrapping

In [33]:
def scrap_article(gid, source):
    start = time.time()
    article = Article(source)
    article.download()
    article.parse()
    article.nlp()
    
    #LOWERCASE
    txt = article.text.lower()
    
    #change nonword to space   
    txt_clean = re.sub(r'[^a-zA-Z]', ' ', txt)
    
    #TOKENISASI , STEM, HAPUS PROPERNOUN, DAN HAPUS STOPWORD
    tagged_sentence = nltk.pos_tag(txt_clean.split())
    
    filtered_sentence=[]
    for initword, tag in tagged_sentence:
        word = ps.stem(initword)
        if len(word)>1 and (word not in stop_words) and tag != 'NNP' and tag != 'NNPS':
            filtered_sentence.append(word)

    #print("TOKENS",type(filtered_sentence))
    
    #COUNT VECTOR: hitung kemunculan tiap kata
    gid_str = str(gid)
    val = dict((Counter(filtered_sentence)))
    
    #masukkan hasil ke dict res dengan id : gid_str dan value: val
    res[gid_str] = val
    end = time.time()
    waktu = end-start
    #print(waktu)
    return val
    

##### Contoh 1 url

In [34]:
sample=data_gdelt['SOURCEURL'][2]
sample
sample_ID = data_gdelt['GlobalEventId'][2]
scrap_article(sample_ID, sample)

{'offici': 4,
 'figur': 1,
 'say': 2,
 'murder': 10,
 'rob': 4,
 'daili': 1,
 'punjab': 6,
 'lahor': 9,
 'averag': 2,
 'person': 6,
 'day': 3,
 'least': 5,
 'peopl': 8,
 'dure': 4,
 'outgo': 2,
 'year': 5,
 'wa': 9,
 'reveal': 1,
 'analysi': 2,
 'polic': 18,
 'data': 6,
 'avail': 2,
 'news': 1,
 'period': 2,
 'januari': 1,
 'novemb': 1,
 'one': 2,
 'certainli': 1,
 'expect': 1,
 'increas': 1,
 'crime': 5,
 'whole': 1,
 'accord': 2,
 'nine': 2,
 'region': 22,
 'capit': 1,
 'citi': 1,
 'total': 1,
 'kill': 12,
 'disput': 1,
 'properti': 4,
 'money': 1,
 'honour': 1,
 'etc': 1,
 'faisalabad': 3,
 'hometown': 1,
 'law': 1,
 'minist': 1,
 'rana': 1,
 'sanaullah': 1,
 'top': 6,
 'gujranwala': 4,
 'secur': 4,
 'second': 3,
 'posit': 10,
 'differ': 2,
 'issu': 1,
 'tech': 1,
 'savvi': 1,
 'provinci': 1,
 'metropoli': 1,
 'stood': 4,
 'third': 4,
 'rawalpindi': 5,
 'calcul': 3,
 'fourth': 5,
 'sheikhupura': 5,
 'fifth': 4,
 'report': 8,
 'incid': 22,
 'sargodha': 4,
 'follow': 3,
 'multan': 5,


In [18]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tryviana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

# Proses Menghasilkan Dict

In [40]:
start2 = time.time()
progress = 0
lenart = 100
for x in range(100):
    global start
    global progress
    progress += 1
    try:
        sample_=data_gdelt['SOURCEURL'][x]
        sample_ID_ = data_gdelt['GlobalEventId'][x]
        aaa = scrap_article(sample_ID_, sample_)
        #print(aaa, type(aaa))
    except:
        print('gagal', sample_)
        pass
    end = time.time()
    print('\r {} percent, {} position, {} per second '.format(str(float(progress / lenart)), str(progress), (1 / (end - start)))) #lets us see how much time is 
end2 = time.time()

print('waktu',end2-start2)

gagal http://nbc16.com/news/local/train-hits-and-kills-teen-boy-in-keizer
 0.01 percent, 1 position, 0.010414103288970706 per second 
gagal http://www.heraldtribune.com/news/20161231/lyons-frontier-is-worthy-suspect-in-fire-alarm-glitch
 0.02 percent, 2 position, 0.009698602418457268 per second 
gagal https://www.thenews.com.pk/print/175863-Official-figures-say-10-murdered-32-robbed-daily-in-Punjab-in-2016
 0.03 percent, 3 position, 0.00883488606646651 per second 
 0.04 percent, 4 position, 0.00860595680933685 per second 
gagal http://www.princegeorgecitizen.com/news/local-news/senior-vancouver-police-officer-charged-with-sex-crimes-and-breach-of-trust-1.5803598
 0.05 percent, 5 position, 0.00793504130859227 per second 
gagal http://www.princegeorgecitizen.com/news/local-news/a-list-of-canadians-who-ran-into-trouble-abroad-in-2016-1.4405000
 0.06 percent, 6 position, 0.007298741218139902 per second 
gagal http://www.princegeorgecitizen.com/news/local-news/a-list-of-canadians-who-ran-in

gagal http://www.9and10news.com/story/34157872/states-re-examine-cybersecurity-after-alleged-russian-hack
 0.55 percent, 55 position, 0.001820486528182716 per second 
 0.56 percent, 56 position, 0.0017982350867868228 per second 
gagal http://www.housepricecrash.co.uk/forum/index.php?/topic/228427-corbyns-ny-speech/&page=2
 0.57 percent, 57 position, 0.0017630386161956685 per second 
gagal http://www.tbo.com/shumlin-issues--pardons-mostly-drug-related-crimes-ap_election414841991fcf4d5b8601c08a2c0f0fe4
 0.58 percent, 58 position, 0.0016931718718942094 per second 
gagal http://www.demingheadlight.com/story/news/2016/12/30/mexican-man-charged-rape-had-19-deportations-removals/96026866/
 0.59 percent, 59 position, 0.001623706825137917 per second 
gagal https://www.balloon-juice.com/2016/12/31/youre-so-vain/
 0.6 percent, 60 position, 0.001585737127979796 per second 
 0.61 percent, 61 position, 0.001545395624139328 per second 
gagal http://www.montrealgazette.com/news/crime-and-justice/calga

In [ ]:
print(type(aaa))

In [22]:
len(data_gdelt)

1392383

# Write hasil ke json

In [25]:
with open('data.json', 'w') as fp:
    json.dump(res, fp)

# Transform Count Vect to Tfidf

### read json file

In [26]:
with open('data.json', 'r') as fp:
    data_counter = json.load(fp)

### add event ID and bag of words to different array

In [27]:
eventID = []
DictVector = []

In [28]:
for k,v in data_counter.items():
    eventID.append(k)
    DictVector.append(v)

### vectorize bag of words from all news

In [29]:
vec = DictVectorizer()
DictArray = vec.fit_transform(DictVector).toarray()

### transform to TFIDF

In [30]:
tfidf_transformer = TfidfTransformer(use_idf=False)
tfidf_transform=tfidf_transformer.transform(DictArray, copy = True)
print(tfidf_transform)
print(DictArray)
print(tfidf_transform.todense())

  (0, 4)	0.06350006350009525
  (0, 12)	0.06350006350009525
  (0, 38)	0.06350006350009525
  (0, 39)	0.06350006350009525
  (0, 52)	0.06350006350009525
  (0, 71)	0.06350006350009525
  (0, 72)	0.06350006350009525
  (0, 106)	0.06350006350009525
  (0, 120)	0.06350006350009525
  (0, 140)	0.06350006350009525
  (0, 146)	0.06350006350009525
  (0, 150)	0.19050019050028574
  (0, 152)	0.06350006350009525
  (0, 154)	0.06350006350009525
  (0, 176)	0.1270001270001905
  (0, 181)	0.19050019050028574
  (0, 199)	0.06350006350009525
  (0, 204)	0.06350006350009525
  (0, 217)	0.06350006350009525
  (0, 232)	0.06350006350009525
  (0, 283)	0.06350006350009525
  (0, 291)	0.06350006350009525
  (0, 319)	0.06350006350009525
  (0, 333)	0.06350006350009525
  (0, 344)	0.06350006350009525
  :	:
  (3, 825)	0.024119153509747387
  (3, 826)	0.024119153509747387
  (3, 827)	0.14471492105848432
  (3, 828)	0.048238307019494775
  (3, 830)	0.024119153509747387
  (3, 831)	0.024119153509747387
  (3, 833)	0.07235746052924216
  (3, 

### header bag of words

In [32]:
vocabulary = vec.vocabulary_

In [33]:
# sort vocabulary by value (for header)
header = sorted(vocabulary.items(), key=operator.itemgetter(1))
header

[('-year-old', 0),
 ('accident', 1),
 ('accidents', 2),
 ('accomplices', 3),
 ('according', 4),
 ('accused', 5),
 ('across', 6),
 ('act', 7),
 ('acted', 8),
 ('action', 9),
 ('added', 10),
 ('address', 11),
 ('age', 12),
 ('ago', 13),
 ('aim', 14),
 ('alarm', 15),
 ('alarms', 16),
 ('alcohol', 17),
 ('alerted', 18),
 ('alleged', 19),
 ('allowed', 20),
 ('already', 21),
 ('also', 22),
 ('amazing', 23),
 ('ambulance', 24),
 ('amiss', 25),
 ('among', 26),
 ('amount', 27),
 ('analysis', 28),
 ('animals', 29),
 ('another', 30),
 ('answered', 31),
 ('anyone', 32),
 ('anything', 33),
 ('application', 34),
 ('appreciation', 35),
 ('approached', 36),
 ('approaching', 37),
 ('area', 38),
 ('around', 39),
 ('arrested', 40),
 ('arrests', 41),
 ('arrival', 42),
 ('arrived', 43),
 ('assaults', 44),
 ('assistance', 45),
 ('assistant', 46),
 ('associates', 47),
 ('asthma', 48),
 ('attack', 49),
 ('attempted', 50),
 ('attorney', 51),
 ('attorneys', 52),
 ('available', 53),
 ('average', 54),
 ('avoided'